In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 이산 확률 분포

- 주사위를 던진하고 할때 확률밀도 함수는 UNIFORM DIST.
- 이산적인 확률분포를 바탕으로 각 확률을 써보자.

 $$ softmax function = P(class = i) - \frac{e^{i}}{\sum{e^i}} $$

- 결국 특정 사건이 일어났을 때 확률을 근사해서 찾는게 목적.

# 소프트 맥스 함수

- 일반적으로 분류 문제에서 CROSSENTROPY, SOFTMAX 사용해야 한다. 
- 따라서 BINARY CROSS ENTROPY와 SIGMOID를 사용하는 경우가 많다. 
- 여러개의 분류 문제에서는 CROSSENTROPY와 SOFTMAX함수를 사용한다.


In [2]:
z = torch.FloatTensor([1, 2, 3])

In [3]:
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
# 전확률의 법칙에 따라서 당연히 1
hypothesis.sum()

tensor(1.)

# Cross Entopy

- 정의: 크로스 엔트로피는 2개의 확률분포가 주어질때, 확률분포가 얼마나 비슷한지 나타낼 수 있는 수치. 

$$ H(P, Q) = - E_{x~P(X)}[\log{Q(x)}] = - \sum_{x}P(x)logQ(x)$$
P, Q 확률 분포가 있을때. 기댓값 



## low level 구현

$$ L = \frac{1}{N}\sum-y\log(\hat{y}) $$

$$y = p(x), log(\hat{y}) = p_{\theta}(x)/Q(x)$$

In [5]:
z = torch.rand(3, 5, requires_grad = True) ## # class: 5, # samples : 3
hypothesis = F.softmax(z, dim =1)
print(hypothesis) # 각 행의 결과가 label 5개에 대한 prediction 값

tensor([[0.1628, 0.2768, 0.1366, 0.1689, 0.2548],
        [0.1151, 0.2175, 0.2166, 0.2281, 0.2227],
        [0.1826, 0.2230, 0.1769, 0.1400, 0.2775]], grad_fn=<SoftmaxBackward0>)


In [6]:
y = torch.randint(5, (3,)).long() 
print(y)# index값을 배출해준다.

tensor([2, 3, 4])


In [7]:
y_one_hot = torch.zeros_like(hypothesis) ## y_one_hot = (3,5)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) ## y = (3,1)

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim = 1).mean()
print(cost)

tensor(1.5836, grad_fn=<MeanBackward0>)


# Cross-entropy Loss with torch.nn.functional

In [9]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.8149, -1.2843, -1.9909, -1.7783, -1.3672],
        [-2.1621, -1.5256, -1.5299, -1.4779, -1.5017],
        [-1.7003, -1.5005, -1.7321, -1.9662, -1.2821]], grad_fn=<LogBackward0>)

In [10]:
# High level 바로 접근 가능.
F.log_softmax(z, dim=1)

tensor([[-1.8149, -1.2843, -1.9909, -1.7783, -1.3672],
        [-2.1621, -1.5256, -1.5299, -1.4779, -1.5017],
        [-1.7003, -1.5005, -1.7321, -1.9662, -1.2821]],
       grad_fn=<LogSoftmaxBackward0>)

In [11]:
# Low level    F.log_softmax
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.5836, grad_fn=<MeanBackward0>)

In [12]:
# High level NLL = negative log likelihood
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.5836, grad_fn=<NllLossBackward0>)

In [13]:
F.cross_entropy(z, y)

tensor(1.5836, grad_fn=<NllLossBackward0>)

- F.cross_entropy를 사용하면, nll_loss와 F.log_softmax를 결합한 결과를 배출 할 수 있다.
- prediction/신경망에서 확률값을 배출해야 하는 경우가 많다.
- F.cross_entropy에서는 자체적으로 확률값을 배출하도록 하는 함수가 포함되어 있기때문에, 확률값을 2번 구하는 일이 일어날 수도 있다.

- 그때 용도에 맞는 함수를 사용하자.

# Training with Low-level Cross Entropy Loss

In [14]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
# 3 class, 4dim, input m
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [15]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

RuntimeError: Expected index [8, 1] to be smaller than self [3, 5] apart from dimension 1

In [19]:
y_train

tensor([2, 2, 2, 1, 1, 1, 0, 0])

In [18]:
y_train.unsqueeze(1)

tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [1],
        [0],
        [0]])

In [17]:
torch.zeros_like(hypothesis)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [16]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    
    hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot *- torch.log((hypothesis))).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
Epoch    0/1000 Cost: 0.366204
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
     

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
Epoch  400/1000 Cost: 0.194423
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
     

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
     

# Training with F.cross_entropy

In [ ]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train) ## CROSS ENTROPY 값을 최소화 함.
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

# High-level Implementation with nn.Module (추가해라)